In [ ]:
# STUDENT NAME: SHRUTHI PASUMARTHI
# STUDENT ID: 801218392
# HOMEWORK 2 QUESTION 1

In [1]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
import pandas as pd 
import math
from datetime import datetime
torch.set_printoptions(edgeitems=2, linewidth=75)

In [2]:
# Obtaining the housing data as given in the sample code
housing = pd.DataFrame(pd.read_csv("C:/Users/Shruthi/Documents/HWs/RTML/hw1/images/Housing.csv")) 
#housing.head() 
housing
housing.shape

(545, 13)

In [3]:
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking','price'] 
Newtrain = housing[num_vars] 
Newtrain

,area,bedrooms,bathrooms,stories,parking,price
0,7420,4,2,3,2,13300000
1,8960,4,4,4,3,12250000
2,9960,3,2,2,2,12250000
3,7500,4,2,2,3,12215000
4,7420,4,1,2,2,11410000
...,...,...,...,...,...,...
540,3000,2,1,1,2,1820000
541,2400,3,1,1,0,1767150
542,3620,2,1,1,0,1750000
543,2910,3,1,1,0,1750000


In [4]:
#Randomly shuffling the rows, to give the model a more random data
t_an = Newtrain.to_numpy()
t_an.shape
row=t_an.shape[0]
column=t_an.shape[1]
print(row)
print(column)
column

545
6


6

In [5]:
# Normalizing the data as per the formula in the textbook 
## P.S. all the operations of the normalization of the dataset are done when the matrix is a numpy array
#Standard Deviations of each column
std0=np.sqrt(np.var(t_an[:,0], axis=0))
std1=np.sqrt(np.var(t_an[:,1], axis=0))
std2=np.sqrt(np.var(t_an[:,2], axis=0))
std3=np.sqrt(np.var(t_an[:,3], axis=0))
std4=np.sqrt(np.var(t_an[:,4], axis=0))
std5=np.sqrt(np.var(t_an[:,5], axis=0))
#Mean of each column
mean0=np.mean(t_an[:,0])
mean1=np.mean(t_an[:,1])
mean2=np.mean(t_an[:,2])
mean3=np.mean(t_an[:,3])
mean4=np.mean(t_an[:,4])
mean5=np.mean(t_an[:,5])
#Iterating to get the normalized values in the dataset
for r in range (row):
    t_an[r, 0]=(t_an[r, 0]-mean0)/std0
    t_an[r, 1]=(t_an[r, 1]-mean1)/std1
    t_an[r, 2]=(t_an[r, 2]-mean2)/std2
    t_an[r, 3]=(t_an[r, 3]-mean3)/std3
    t_an[r, 4]=(t_an[r, 4]-mean4)/std4
    t_an[r, 5]=(t_an[r, 5]-mean5)/std5

print(std0, std1, std2, std3, std4, std5)
print(mean0, mean1, mean2, mean3, mean4, mean5)
print(t_an)

2168.149153321305 0.7373864268686331 0.5020084231005496 0.8666962328697991 0.8607949418951395 1868722.8281312082
5150.54128440367 2.9651376146788992 1.2862385321100918 1.8055045871559634 0.6935779816513762 4766729.247706422
[[ 1  1  1  1  1  4]
 [ 1  1  5  2  2  4]
 [ 2  0  1  0  1  4]
 ...
 [ 0 -1  0  0  0 -1]
 [-1  0  0  0  0 -1]
 [ 0  0  0  0  0 -1]]


In [6]:
#Splitting the dataset into the training and validation sets, where 80% of the data is the training dataset and the remaining 
# 20% is the validation set

train_size = int(0.8 * row)
indices = np.random.permutation(t_an.shape[0])
indices.shape
training_idx, val_idx = indices[:train_size], indices[train_size:]
t_anf = t_an.astype(np.float32)
train_data, val_data = t_anf[training_idx,:], t_anf[val_idx,:]
print(train_data.shape)
print(val_data.shape)

t_a1, t_a2, t_a3, t_a4, t_a5, train_labels = train_data.T
t_t = (t_a1, t_a2, t_a3, t_a4, t_a5)
train_features = np.vstack(t_t)
train_features = np.transpose(train_features)
train_features = torch.tensor(train_features)
train_labels = torch.tensor(train_labels)
train_labels = train_labels.unsqueeze(1)
print(train_features.shape)
print(train_labels.shape)


v_a1, v_a2, v_a3, v_a4, v_a5, val_labels = val_data.T
v_t = (v_a1, v_a2, v_a3, v_a4, v_a5)
val_features = np.vstack(v_t)
val_features = np.transpose(val_features)
val_features = torch.tensor(val_features)
val_labels = torch.tensor(val_labels)
val_labels = val_labels.unsqueeze(1)
print(val_features.shape)
print(val_labels.shape)

(436, 6)
(109, 6)
torch.Size([436, 5])
torch.Size([436, 1])
torch.Size([109, 5])
torch.Size([109, 1])


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# The first model, with only one hidden block

model = nn.Sequential(
            nn.Linear(5,8),
            nn.ReLU(), 
            nn.LogSoftmax(dim=1))

learning_rate = 1e-3

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

out = model(train_features[0,:].unsqueeze(0))


loss_fn = nn.MSELoss()

n_epochs = 200
correct = 0
total = 0

for epoch in range(n_epochs):
    
        out = model(train_features)
        loss = loss_fn(out,train_labels)
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch==0 or epoch % 10 ==0:
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print(dt_string, "Epoch: %d, Loss: %f" % (epoch, float(loss)))
        _, predicted = torch.max(out, dim=1)
        total += val_labels.shape[0]
        correct += int((predicted == val_labels).sum())
        
print("Accuracy: %f" % (correct / total))
        


04/03/2022 02:17:29 Epoch: 0, Loss: 5.475407
04/03/2022 02:17:29 Epoch: 10, Loss: 5.473634
04/03/2022 02:17:29 Epoch: 20, Loss: 5.471878
04/03/2022 02:17:29 Epoch: 30, Loss: 5.470138
04/03/2022 02:17:29 Epoch: 40, Loss: 5.468415
04/03/2022 02:17:29 Epoch: 50, Loss: 5.466708
04/03/2022 02:17:29 Epoch: 60, Loss: 5.465018
04/03/2022 02:17:29 Epoch: 70, Loss: 5.463344
04/03/2022 02:17:29 Epoch: 80, Loss: 5.461683
04/03/2022 02:17:29 Epoch: 90, Loss: 5.460039
04/03/2022 02:17:29 Epoch: 100, Loss: 5.458409
04/03/2022 02:17:29 Epoch: 110, Loss: 5.456793
04/03/2022 02:17:29 Epoch: 120, Loss: 5.455192
04/03/2022 02:17:29 Epoch: 130, Loss: 5.453603
04/03/2022 02:17:29 Epoch: 140, Loss: 5.452028
04/03/2022 02:17:29 Epoch: 150, Loss: 5.450468
04/03/2022 02:17:29 Epoch: 160, Loss: 5.448921


C:\Users\Shruthi\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([436, 1])) that is different to the input size (torch.Size([436, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


04/03/2022 02:17:29 Epoch: 170, Loss: 5.447390
04/03/2022 02:17:29 Epoch: 180, Loss: 5.445871
04/03/2022 02:17:29 Epoch: 190, Loss: 5.444367
Accuracy: 17.350275


In [8]:
#To calculate the accuracy of the validation dataset using the model with one hidden layer 

correct = 0
total = 0

with torch.no_grad():
    
        outputs = model(val_features)
        loss = loss_fn(outputs,val_labels)
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
        _, predicted = torch.max(outputs, dim=1)
        total += val_labels.shape[0]
        correct += int((predicted == val_labels).sum())
        
print("Accuracy: %f" % (correct / total))

Epoch: 199, Loss: 5.624659
Accuracy: 4.467890


C:\Users\Shruthi\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([109, 1])) that is different to the input size (torch.Size([109, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

#The second model, with 3 hidden blocks

model = nn.Sequential(
            nn.Linear(5,16),
            nn.ReLU(),
            nn.Linear(16,8),
            nn.ReLU(),
            nn.Linear(8,2),
            nn.ReLU(),
            nn.LogSoftmax(dim=1))

learning_rate = 1e-3

optimizer = optim.SGD(model.parameters(), lr=learning_rate)


out = model(train_features[0,:].unsqueeze(0))


loss_fn = nn.MSELoss()

n_epochs = 200
correct = 0
total = 0

for epoch in range(n_epochs):
    
        out = model(train_features)
        loss = loss_fn(out,train_labels)
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch==0 or epoch % 10 ==0:
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print(dt_string, "Epoch: %d, Loss: %f" % (epoch, float(loss)))
        _, predicted = torch.max(out, dim=1)
        total += val_labels.shape[0]
        correct += int((predicted == val_labels).sum())
        
print("Accuracy: %f" % (correct / total))

04/03/2022 02:17:29 Epoch: 0, Loss: 1.099316
04/03/2022 02:17:29 Epoch: 10, Loss: 1.099034
04/03/2022 02:17:29 Epoch: 20, Loss: 1.098759
04/03/2022 02:17:29 Epoch: 30, Loss: 1.098490
04/03/2022 02:17:29 Epoch: 40, Loss: 1.098228
04/03/2022 02:17:29 Epoch: 50, Loss: 1.097971
04/03/2022 02:17:29 Epoch: 60, Loss: 1.097720
04/03/2022 02:17:29 Epoch: 70, Loss: 1.097474
04/03/2022 02:17:29 Epoch: 80, Loss: 1.097234
04/03/2022 02:17:29 Epoch: 90, Loss: 1.096999
04/03/2022 02:17:29 Epoch: 100, Loss: 1.096770
04/03/2022 02:17:29 Epoch: 110, Loss: 1.096545
04/03/2022 02:17:29 Epoch: 120, Loss: 1.096326
04/03/2022 02:17:29 Epoch: 130, Loss: 1.096112
04/03/2022 02:17:29 Epoch: 140, Loss: 1.095902
04/03/2022 02:17:29 Epoch: 150, Loss: 1.095697
04/03/2022 02:17:29 Epoch: 160, Loss: 1.095497


C:\Users\Shruthi\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([436, 1])) that is different to the input size (torch.Size([436, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


04/03/2022 02:17:29 Epoch: 170, Loss: 1.095301
04/03/2022 02:17:29 Epoch: 180, Loss: 1.095110
04/03/2022 02:17:29 Epoch: 190, Loss: 1.094923
Accuracy: 320.000000


In [10]:
#To calculate the accuracy of the validation dataset using the model with three hidden layers 

correct = 0
total = 0

with torch.no_grad():
    
        outputs = model(val_features)
        loss = loss_fn(outputs,val_labels)
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
        _, predicted = torch.max(outputs, dim=1)
        total += val_labels.shape[0]
        correct += int((predicted == val_labels).sum())
        
print("Accuracy: %f" % (correct / total))

Epoch: 199, Loss: 1.228506
Accuracy: 80.000000


C:\Users\Shruthi\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([109, 1])) that is different to the input size (torch.Size([109, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
